In [28]:
from datasets import Dataset
import pandas as pd
import numpy as np

In [29]:
data = pd.read_csv('../data/clean/sustainability-report-2020-clean-BST_KK-fixed-with_starts.csv', sep=';')

In [30]:
# Randomly copy half of the rows to a new dataframe
data_sample = data.sample(frac=0.5, random_state=1)
data_sample = data_sample.reset_index(drop=True)
# Set all values in the fLABEL column to NOTOK
data_sample['fLABEL'] = 'NOT_OK'
# Shuffle the context column
data_sample['context'] = np.random.permutation(data_sample['context'])
# Get starting positions
for i in range(len(data_sample)):
    text = data_sample["context"][i]
    answer = data_sample["answer"][i]
    answer_start = text.find(answer)
    data_sample.loc[i, "answer_start"] = int(answer_start)

# Set label to OK for all rows where the answer_start is not -1
data_sample.loc[data_sample['answer_start'] != -1, 'fLABEL'] = 'OK'

# Concatenate the original dataframe with the modified sample
data = pd.concat([data, data_sample], ignore_index=True)

In [31]:
# Create a new column containing dictionaries
data['answers'] = data.apply(lambda x: {'text': [x['answer']] * 3, 'answer_start': [x['answer_start']] * 3}, axis=1)
data["id"] = data.index

In [32]:
data.reset_index(drop=True, inplace=True)
data = data.drop(columns=['answer_start', 'answer', 'fLABEL'])

In [33]:
train_df = data.sample(frac=0.8, random_state=1)
test_df = data.drop(train_df.index)

In [34]:
train_ds = Dataset.from_pandas(train_df, split="train", preserve_index=False)
test_ds = Dataset.from_pandas(test_df, split="test", preserve_index=False)

In [35]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [36]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [37]:
train_dataset = train_ds.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_ds.column_names,
)
len(train_ds), len(train_dataset)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

(222, 222)

In [38]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]
    
    inputs["example_id"] = example_ids
    return inputs

In [39]:
validation_dataset = test_ds.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_ds.column_names,
)
len(test_ds), len(validation_dataset)

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

(55, 55)

In [40]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [41]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

In [42]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [43]:
trainer.train()

c:\Users\rjutr\miniconda3\envs\project_ds\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_runtime': 0.2979, 'eval_samples_per_second': 184.631, 'eval_steps_per_second': 13.428, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_runtime': 0.298, 'eval_samples_per_second': 184.58, 'eval_steps_per_second': 13.424, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_runtime': 0.3053, 'eval_samples_per_second': 180.164, 'eval_steps_per_second': 13.103, 'epoch': 3.0}
{'train_runtime': 13.036, 'train_samples_per_second': 51.089, 'train_steps_per_second': 3.222, 'train_loss': 0.5904768989199684, 'epoch': 3.0}


TrainOutput(global_step=42, training_loss=0.5904768989199684, metrics={'train_runtime': 13.036, 'train_samples_per_second': 51.089, 'train_steps_per_second': 3.222, 'train_loss': 0.5904768989199684, 'epoch': 3.0})

In [44]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

Found cached dataset squad (C:/Users/rjutr/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
small_eval_set = test_ds.select(range(50))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

# Add length to the answers in the dataset
for i, example in enumerate(small_eval_set):
    small_eval_set[i]["answers"]["len"] = len(example["answers"]["text"][0])
    

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_ds.column_names,
)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [46]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [47]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [48]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [49]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [50]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [51]:
theoretical_answers

[{'id': 1,
  'answers': {'answer_start': [69, 69, 69], 'text': ['49%', '49%', '49%']}},
 {'id': 3,
  'answers': {'answer_start': [16, 16, 16], 'text': ['31%', '31%', '31%']}},
 {'id': 7, 'answers': {'answer_start': [69, 69, 69], 'text': ['1', '1', '1']}},
 {'id': 8,
  'answers': {'answer_start': [68, 68, 68], 'text': ['58%', '58%', '58%']}},
 {'id': 22,
  'answers': {'answer_start': [69, 69, 69],
   'text': ['knowledge and lifelong learning',
    'knowledge and lifelong learning',
    'knowledge and lifelong learning']}},
 {'id': 24,
  'answers': {'answer_start': [69, 69, 69],
   'text': ['a higher quality of life of the wider society',
    'a higher quality of life of the wider society',
    'a higher quality of life of the wider society']}},
 {'id': 26,
  'answers': {'answer_start': [69, 69, 69],
   'text': ['local banking and environmental legislation',
    'local banking and environmental legislation',
    'local banking and environmental legislation']}},
 {'id': 30,
  'answers': {

In [52]:
import evaluate

metric = evaluate.load("squad")

In [58]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)